In [1]:
!pip install sentence_transformers
!pip install wikipedia
# !wget -c "https://figshare.com/ndownloader/files/13774556"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 4.7 MB 41.3 MB/s 
     |████████████████████████████████| 1.3 MB 58.2 MB/s 
     |████████████████████████████████| 101 kB 10.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c7fc662b6bf381aa82fd51a64370dd80f1a5800a9090dc6a189695a5935679d3
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=b8f33702bd9d2af4fe713b7fcb35bcfb26806c67a25208f675c4ae050e847e2b
  Stored in directory: /root/

In [2]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
from nltk.util import ngrams
import wikipedia as wiki
import gensim
import re
import nltk
from nltk import tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


model = SentenceTransformer('bert-base-nli-mean-tokens')
nlp = en_core_web_sm.load()
nltk.download('punkt')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Use n-grams for search

In [14]:
def keywords(sentence):

    doc = nlp(sentence)

    results = ([(X.text, X.label_)[0] for X in doc.ents])
    return list(set(results))

In [4]:
def clean_wiki_text(text):
    text = re.sub(r'==.+==', '.', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\[[0-9]+\]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\. \.', '.', text)
    return text

In [5]:
def content(claim, results):
    sentences = []
    found=[]
    for i in results:
        try :
            current_page = wiki.page(i)
            if current_page not in found:
                found.append(current_page)
        except :
            continue
    titles=[i.title for i in found]
    titles=[i[0] for i in topNSimilar(claim, titles)]
    for i in found:
        if i.title not in titles:
            found.remove(i)

    for i in found:
        current_content = i.content
        sentences.extend(tokenize.sent_tokenize(clean_wiki_text(current_content)))
    return sentences

In [6]:
def topNSimilar(claim, sentList, n=5):
    distList=[]
    for i in sentList:
        document_term_matrix = TfidfVectorizer().fit_transform([i, claim])
        dist = document_term_matrix * document_term_matrix.transpose()
        distance = dist.toarray()[0][1]

        if len(distList)<=n:
            distList.append([i, distance])
        else:
            distList=sorted(distList, key=lambda x: x[1], reverse=True)
            if distance>distList[-1][1]:
                distList.pop()
                distList.append([i, distance])
    return sorted(distList, key=lambda x: x[1], reverse=True)

In [7]:
def topNBert(claim, res):
    topNFacts=[i[0] for i in res]
    topNScore=[cosine_similarity( [model.encode(claim)], [model.encode(i)] )[0][0] for i in topNFacts]
    topN=zip(topNFacts, topNScore)
    return [list(i) for i in list(topN)]

In [11]:
claim = 'Biden is the president of the united states'

In [12]:
final_results = topNBert(claim, topNSimilar(claim, content(claim, keywords(claim)), 5))

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [18]:
final_text_results = []
for i in final_results :
  final_text_results.append(i[0])
final_text_results

['The president is the commander-in-chief of the United States Armed Forces and appoints its leaders, the secretary of defense and the Joint Chiefs of Staff.',
 'United States, from the BBC News Key Development Forecasts for the United States from International FuturesGovernmentOfficial U.S. Government Web Portal Gateway to government sites House Official site of the United States House of Representatives Senate Official site of the United States Senate White House Official site of the president of the United States Supreme Court Official site of the Supreme Court of the United StatesHistoryHistorical Documents Collected by the National Center for Public Policy Research U.S. National Mottos: History and Constitutionality Analysis by the Ontario Consultants on Religious Tolerance USA Collected links to historical dataMapsNational Atlas of the United States Official maps from the U.S. Department of the Interior Wikimedia Atlas of the United States Geographic data related to United States